In [8]:
import cloudpickle
import numpy as np
import pandas as pd
from janitor import clean_names

# Ruta y archivo
ruta_proyecto = 'C:/Users/Alfonso/OneDrive/Documentos_/Formación/Bootcamp_DS/Repo_DS/Proyect_Break_ML/00_PROYECTO_ML'
nombre_fichero_datos = 'marketing_campaign.xlsx'
ruta_completa = f'{ruta_proyecto}/02_Datos/01_Originales/{nombre_fichero_datos}'

# Cargar datos
df_original = pd.read_excel(ruta_completa, index_col=0)
df = df_original.copy()

df = clean_names(df).drop_duplicates()

# Preprocesamiento
dicc_status = {
    'Together': 2,
    'Divorced': 1,
    'Married': 2,
    'Single': 1,
    'Widow': 1,
    'OTROS': 1
}
df['status_members'] = df['marital_status'].map(dicc_status)
df['household_members'] = df['status_members'] + df['kidhome'] + df['teenhome']
df['total_cmp'] = df.filter(like='accepted').apply(pd.to_numeric, errors='coerce').sum(axis=1)
train_set_numeric = df.filter(like='cmp').apply(pd.to_numeric, errors='coerce')
df = df.drop(columns=['status_members'])

# Variables finales
variables_finales = ['household_members', 'income', 'mntgoldprods', 'mntmeatproducts',
                    'mntwines', 'numcatalogpurchases', 'numstorepurchases', 'numwebvisitsmonth',  'total_cmp']
X = df[variables_finales]

# Cargar modelo
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'
ruta_pipe_ejecucion = f'{ruta_proyecto}/04_Modelos/{nombre_pipe_ejecucion}'

with open(ruta_pipe_ejecucion, mode='rb') as file:
    pipe_ejecucion = cloudpickle.load(file)


probabilidades = pipe_ejecucion.predict_proba(X)[:, 1]

df_original = clean_names(df_original).drop_duplicates() 
df_original['prediccion'] = probabilidades
df_original['prediccion'] = df_original['prediccion'].apply(lambda x: round(x,2)) 

nombre_df_predict = 'df_predict_final.csv'
ruta_df_predict = f'{ruta_proyecto}/02_Datos/03_Trabajo/{nombre_df_predict}'
df_original.to_csv(ruta_df_predict)


In [10]:
df_original

,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,recency,mntwines,mntfruits,...,acceptedcmp3,acceptedcmp4,acceptedcmp5,acceptedcmp1,acceptedcmp2,complain,z_costcontact,z_revenue,response,prediccion
ID,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,...,0,0,0,0,0,0,3,11,1,0.79
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,...,0,0,0,0,0,0,3,11,0,0.13
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,...,0,0,0,0,0,0,3,11,0,0.18
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,...,0,0,0,0,0,0,3,11,0,0.12
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,...,0,0,0,0,0,0,3,11,0,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9432,1977,Graduation,Together,666666.0,1,0,2013-06-02,23,9,14,...,0,0,0,0,0,0,3,11,0,0.28
10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,43,...,0,0,0,0,0,0,3,11,0,0.27
7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,48,...,0,1,0,0,0,0,3,11,0,0.77
